In [1]:
import numpy as np
import math as m
from matplotlib import pyplot as plt
import random as r
from  matplotlib.animation import FuncAnimation
import sys
import stats
import pickle
from scipy import interpolate

In [2]:
## Takes array of data and returns it window averaged
def rolling_avg(arr, window):
    to_return = []
    for i in range(len(arr)):
        lower = int(i-window/2)
        upper = int(i+window/2)
        if lower < 0: lower = 0
        if upper >= len(arr): upper = len(arr)-1
        to_return.append(sum(arr[lower:upper])/(upper-lower))          
    return to_return

In [3]:
class Metropolis():
    
    def __init__(self, L, T=1, initial_state=0, B=False):
        if type(initial_state)==int:
            self.grid = np.ones(shape=[L, L])
            for i in range(L):
                self.grid[i] = np.random.choice([-1, 1], size=L)
        else:
            self.grid = initial_state
        self.L = L
        self.T = T
        self.B = 1/T
        if B:
            self.T = self.B
            self.B = T
        self.N = L*L
        self.energy = self.calc_energy()
        self.energy_history = [self.energy]
        self.grid_history = [np.copy(self.grid)]
        self.M_history = [np.sum(self.grid)]
        
        # This is for a L x L lattice
        x,y=np.indices((L,L))
        red=np.logical_and((x+y) % 2==0   , np.logical_and(x<L-1,y<L-1))
        red[L-1,L-1]=True
        blue=np.logical_and((x+y) % 2==1   , np.logical_and(x<L-1,y<L-1))
        green=np.logical_and((x+y) % 2==0   , np.logical_or(x==L-1,y==L-1))
        green[L-1,L-1]=False
        yellow=np.logical_and((x+y) % 2==1   , np.logical_or(x==L-1,y==L-1))
        self.red = red
        self.blue = blue
        self.green = green
        self.yellow = yellow
        self.colors = [red, blue, green, yellow]
        
        self.J_right = np.ones(shape=[L, L])
        self.J_up = np.ones(shape=[L,L])
        for i in range(L):
            self.J_right[i] = np.random.choice([-1, 1], size=L)
            self.J_up[i] = np.random.choice([-1, 1], size=L)
        return
    
    def P(self, E):
        try:
            return m.exp(-self.B*E)
        except:
            print(f"E = {-self.B*E}")
            raise Exception("E too big for m.exp()")
    
    def run(self, sweeps, to_print=False):
        for sweep in range(sweeps):
            for color in self.colors:
                delta_Es = self.find_delta_Es(color)
                to_flip = np.random.random(size=delta_Es.shape) < 1/(1+np.exp(self.B*delta_Es))
                self.grid[color] = [-spin if flip else spin for spin, flip in zip(self.grid[color], to_flip)]
                self.energy += np.sum(delta_Es*to_flip)
            self.energy_history.append(self.energy)
            self.M_history.append(np.sum(self.grid))
            self.grid_history.append(np.copy(self.grid))
        return self.grid
    
    def find_delta_Es(self, color):
        orig = self.grid[color]
        delta_Es = np.zeros(shape=self.grid[color].shape)
        delta_Es += 2*orig * np.roll(self.grid, 1, axis=0)[color] * self.J_up[color]
        delta_Es += 2*orig * np.roll(self.grid, -1, axis=0)[color] * np.roll(self.J_up, -1, axis=0)[color]
        delta_Es += 2*orig * np.roll(self.grid, 1, axis=1)[color] * np.roll(self.J_right, 1, axis=0)[color]
        delta_Es += 2*orig * np.roll(self.grid, -1, axis=1)[color] * self.J_right[color]
        return delta_Es
    
    def flip_point(self, pt):
        delta_e = self.grid[pt]
        
    def left(self, point):
        if point[1] == 0:
            return [point[0], self.L-1]
        return [point[0], point[1]-1]
    
    def right(self, point):
        if point[1] == self.L-1:
            return [point[0], 0]
        return [point[0], point[1]+1]
    
    def up(self, point):
        if point[0] == 0:
            return [self.L-1, point[1]]
        return [point[0]-1, point[1]]
    
    def down(self, point):
        if point[0] == self.L-1:
            return [0, point[1]]
        return [point[0]+1, point[1]]
    
    def calc_energy(self, grid=0):
        if type(grid) == int: grid = self.grid
        energy = 0
        for y in range(self.L):
            for x in range(self.L):
                energy += grid[y, x] * self.E_up((y, x))
        return int(energy)
    
    def E_up(self, pt, grid=0, point=False):
        if type(grid) == int: grid = self.grid
        e = 0
        e += -grid[self.up(pt)[0], self.up(pt)[1]] * self.J_up
        e += -grid[self.right(pt)[0], self.right(pt)[1]] * 
        if point:
            e += -grid[self.down(pt)[0], self.down(pt)[1]]
            e += -grid[self.left(pt)[0], self.left(pt)[1]]
        return e
    
    def get_grid(self):
        return self.grid
    
    def get_energy_history(self):
        return self.energy_history
    
    def get_grid_history(self):
        return self.grid_history
    
    def get_M_history(self):
        return self.M_history

In [ ]:
def run_simulated_annealing(Bs):
    